In [24]:
import random
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
import google.generativeai as palm
import pandas as pd
import csv
import time
import os

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/a1232953/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/a1232953/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/a1232953/nltk_data...


True

In [4]:
palm.configure(api_key='YOUR_API_KEY')

In [5]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


In [6]:
def talktomodel(prompt):
    completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
    )

    return(completion.result)

In [7]:
def add_to_csv(i, task, inputID, inputText, pertMethod, pertText, origOut, pertOut, origTime, pertTime):
    csvName = 'CreatedFunctions_Iteration' + str(i) + '_' + task + '_PaLMAPI.csv'
    
    if not os.path.isfile(csvName) or os.path.getsize(csvName) == 0:
        with open(csvName, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(['InputTextID', 'InputText', 'PerturbationID', 'PerturbedText', 'OriginalOutput', 'PerturbedOutput', 'OriginalTime', 'PerturbedTime'])
    
    with open(csvName, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([inputID, inputText, pertMethod, pertText, origOut, pertOut, origTime, pertTime])

In [92]:
def introduce_typos(input_string): #character-level
    typo_probability = 0.1
    typoed_string = ""
    for char in input_string:
        if random.uniform(0, 1) < typo_probability:
            typoed_string += chr(random.randint(97, 122))#random lowercase character
        else:
            typoed_string += char
    return typoed_string

In [93]:
def delete_characters(input_text): #character-level
    deletion_probability = 0.1
    deleted_text = ''
    for char in input_text:
        if random.uniform(0, 1) >= deletion_probability:
            deleted_text += char
    return deleted_text

In [94]:
def shuffle_characters(input_text): #character-level
    shuffle_probability = 0.1
    words = input_text.split()
    new_words = []
    
    for word in words:
        if len(word) <= 3 or random.uniform(0, 1) > shuffle_probability:
            new_words.append(word)
        else:
            first_char = word[0]
            last_char = word[-1]
            middle_chars = list(word[1:-1])
            random.shuffle(middle_chars)
            shuffled_word = first_char + ''.join(middle_chars) + last_char
            new_words.append(shuffled_word)
            
    return ' '.join(new_words)

In [95]:
def add_characters(input_string): #character-level
    typo_probability = 0.1
    typoed_string = ""
    for char in input_string:
        if random.uniform(0, 1) < typo_probability:
            typoed_string += chr(random.randint(97, 122))
            typoed_string += char
        else:
            typoed_string += char
    return typoed_string

In [96]:
def to_leet(input_string): #character-level
    
    leet_mapping = {
        'a': '4',
        'e': '3',
        'i': '1',
        'o': '0'
    }
    leet_words = []
    
    leet_probability = 0.2
    
    words = input_string.split()
    for word in words:
        if random.uniform(0, 1) < leet_probability:
            leet_text = ''
            for char in word:
                lowercase_char = char.lower()
                if lowercase_char in leet_mapping:
                    leet_text += leet_mapping[lowercase_char]
                else:
                    leet_text += char
                    
            leet_words.append(leet_text)
            
        else:
            leet_words.append(word)
            
    return ' '.join(leet_words)

In [97]:
def add_spaces(input_string): #character-level
    space_probability = 0.3
    spaced_string = ""
    for char in input_string:
        if random.uniform(0, 1) < space_probability:
            spaced_string += ' '
            spaced_string += char
        else:
            spaced_string += char
    return spaced_string

In [98]:
def add_words(input_text): #word-level
    new_word_probability = 0.2
    random_words = ["apple", "grape", "banana", "pear"]
    words = input_text.split()
    new_words = []
    for word in words:
        if random.uniform(0, 1) < new_word_probability:
            new_words.append(word)
            new_words.append(random.choice(random_words))
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [99]:
def swap_characters(input_text): #character-level
    swap_probability = 0.2
    words = input_text.split()
    new_words = []
    
    for word in words:
        if len(word) <= 3 or random.uniform(0, 1) > swap_probability:
            new_words.append(word)
        else:
            first_char = word[0]
            last_char = word[-1]
            middle_chars = list(word[1:-1])
            char_no = range(len(middle_chars))
            c1, c2 = random.sample(char_no, 2)
            middle_chars[c1], middle_chars[c2] = middle_chars[c2], middle_chars[c1]
            swapped_word = first_char + ''.join(middle_chars) + last_char
            new_words.append(swapped_word)
            
    return ' '.join(new_words)

In [100]:
def antonym_replacement(input_text): #word-level
    words = input_text.split()
    new_words = []
    for word in words:
        antonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                if lemma.antonyms():
                    antonyms.append(lemma.antonyms()[0].name())
        if antonyms:
            new_word = random.choice(antonyms)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [101]:
def synonym_replacement(input_text): #word-level
    words = input_text.split()
    new_words = []
    for word in words:
        synonyms = []
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if synonyms:
            new_words.append(random.choice(synonyms))
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [73]:
def replace_random_sentence(input_text): #sentence-level
    random_sentences = ["Lorem ipsum dolor sit amet.", "Sit modi ipsam quo illum commodi et quia quisquam.", "Aut illo quibusdam sed voluptatum perspiciatis ut minus obcaecati."]
    sentences = sent_tokenize(input_text)

    for i in range(1,4):
        sent_index = random.randint(0, len(sentences) - 1)
        random_sentence = random.choice(random_sentences)
        sentences[sent_index] = random_sentence

    return ' '.join(sentences)

In [102]:
def remove_random_sentence(input_text): #sentence-level
    sentences = sent_tokenize(input_text)

    for i in range(1,4):
        sent_index = random.randint(0, len(sentences) - 1)
        del sentences[sent_index]

    return ' '.join(sentences)

In [103]:
prompt_mapping = {
    "sentiment analysis": "Analyse the sentiment of this text as positive, negative or neutral.",
    "text summarization": "Summarize this text in five sentences",
    "information retrieval": "List the top 10 information from this text.",
    "toxicity detection": "Check whether this text contains toxic or spam content and say yes, no or unknown. Also provide reasons.",
    "news classification": "Categorize the news text into the following categories: World, Sports, Business, Science/Technology."
}

In [104]:
def process_row(row, iteration):
    # add_to_csv(iteration, task, inputID, inputText, pertMethod, pertText, origOut, pertOut, origTime, pertTime)
    task = row['task']
    input_text = row['input']
    inputID = row['inputID']
    
    if task in prompt_mapping:
        
        if task == "sentiment analysis" or task == "toxicity detection" or task == "news classification":
            prompt = prompt_mapping[task]
            
            orig_new_text = input_text + " " + prompt
            start_time = time.time()
            origOut = talktomodel(orig_new_text)
            end_time = time.time()
            origTime = end_time - start_time
        
            perturbed_text = introduce_typos(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Introducing Typos"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = delete_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Deleting Characters"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = synonym_replacement(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Replacing Synonyms"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = add_words(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Adding random words"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = to_leet(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Converting to l33t format"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = shuffle_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Shuffling characters in each word"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = add_spaces(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Adding spaces in text"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = add_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Adding random characters"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = swap_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Swapping two random characters in each word"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = antonym_replacement(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Replacing words with their antonyms"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        elif task == "text summarization" or task == "information retrieval":
            prompt = prompt_mapping[task]
            
            orig_new_text = input_text + " " + prompt
            start_time = time.time()
            origOut = talktomodel(orig_new_text)
            end_time = time.time()
            origTime = end_time - start_time
        
            perturbed_text = introduce_typos(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Introducing Typos"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = delete_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Deleting Characters"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = synonym_replacement(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Replacing Synonyms"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
            perturbed_text = add_words(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Adding random words"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
     
            perturbed_text = replace_random_sentence(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Replacing random sentences"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = remove_random_sentence(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Removing random sentences"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = to_leet(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Converting to l33t format"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = add_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Adding random characters"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = antonym_replacement(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Replacing words with their antonyms"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
            perturbed_text = shuffle_characters(input_text)
            pert_new_text = perturbed_text + " " + prompt
            pertMethod = "Shuffling characters in each word"
            start_time = time.time()
            pertOut = talktomodel(pert_new_text)
            end_time = time.time()
            pertTime = end_time - start_time
            add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
    else:
        # task = question answering
        orig_new_text = input_text
        start_time = time.time()
        origOut = talktomodel(orig_new_text)
        end_time = time.time()
        origTime = end_time - start_time
        
        pert_new_text = introduce_typos(input_text)
        # print(pert_new_text)
        pertMethod = "Introducing Typos"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
        pert_new_text = delete_characters(input_text)
        pertMethod = "Deleting Characters"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
        pert_new_text = synonym_replacement(input_text)
        pertMethod = "Replacing Synonyms"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
        pert_new_text = add_words(input_text)
        pertMethod = "Adding random words"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
        
        pert_new_text = to_leet(input_text)
        pertMethod = "Converting to l33t format"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        pert_new_text = shuffle_characters(input_text)
        pertMethod = "Shuffling characters in each word"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        pert_new_text = add_spaces(input_text)
        pertMethod = "Adding spaces in text"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        pert_new_text = add_characters(input_text)
        pertMethod = "Adding random characters"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        pert_new_text = swap_characters(input_text)
        pertMethod = "Swapping two random characters in each word"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)
            
        pert_new_text = antonym_replacement(input_text)
        pertMethod = "Replacing words with their antonyms"
        start_time = time.time()
        pertOut = talktomodel(pert_new_text)
        end_time = time.time()
        pertTime = end_time - start_time
        add_to_csv(iteration, task, inputID, orig_new_text, pertMethod, pert_new_text, origOut, pertOut, origTime, pertTime)

In [130]:
def main():
    csv_file_path = './ExecutionModule/InputsByTask/ir_inputs.csv'  
    df = pd.read_csv(csv_file_path)

    for i in range(10,11):
        for index, row in df.iterrows():
            if i == 10 and index <= 88:
                continue
            print(i, index)
            process_row(row,i)
            time.sleep(1)

In [131]:
main()

10 89
10 90
10 91
10 92
10 93
10 94
10 95
10 96
10 97
10 98
10 99
